In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI 
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

c:\Users\Lenovo\anaconda3\envs\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm.invoke("Tell me about yourself")

AIMessage(content='I\'m a large language model, trained by Google.  I don\'t have a physical body, a personal life, or emotions.  My purpose is to process information and respond to a wide range of prompts and questions. I do this by drawing on a massive dataset of text and code that I was trained on.  This dataset includes things like books, articles, code, and conversations.\n\nWhile I can generate human-like text in response to a wide range of prompts and questions, it\'s important to remember that I don\'t actually "understand" things in the same way a human does.  I don\'t have beliefs, opinions, or consciousness.  Instead, I identify patterns in the data I was trained on and use those patterns to generate text that is relevant to the given prompt.\n\nI can be used for a variety of tasks, such as:\n\n* **Answering questions:** I can provide information on a wide range of topics.\n* **Generating creative content:** I can write stories, poems, articles, and other types of text.\n* *

## LLM Chain

In [33]:

from langchain.prompts import PromptTemplate

prompt=PromptTemplate(input_variables=["topic"],template="Explain {topic} in a single sentence.")
llm_chain=prompt|llm
response=llm_chain.invoke({"topic":"quantum computing"})

In [22]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

#Other types withsummary,with k window memory



store={}

def get_session_history(session_id:str):
    if session_id not in store:
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]


chain=RunnableWithMessageHistory(llm,get_session_history)

res1=chain.invoke("Hi I am abhinav",config={"configurable": {"session_id": "1"}})
res2=chain.invoke("Hi I am john",config={"configurable": {"session_id": "2"}})





In [29]:
chain.invoke("What is my name",config={"configurable": {"session_id": "1"}}).content




'You told me your name is Abhinav.  Do you have any other questions?\n'

In [27]:
chain.invoke("What is my name",config={"configurable": {"session_id": "2"}}).content

'You told me your name is John.\n'

In [51]:
from langchain_core.output_parsers.list import CommaSeparatedListOutputParser
output_parser=CommaSeparatedListOutputParser()
format_instructions=output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {topic}.\n\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": format_instructions}
)


chain=prompt|llm|output_parser

result=chain.invoke({"topic":"machine learning"})

result


['Supervised Learning',
 'Unsupervised Learning',
 'Reinforcement Learning',
 'Semi-supervised Learning',
 'Deep Learning']

### Structred Output

In [49]:
from langchain.output_parsers.structured import StructuredOutputParser, ResponseSchema


# Define the structure you want
response_schemas = [
    ResponseSchema(name="name", description="The name of the programming language"),
    ResponseSchema(name="year", description="Year the language was created"),
    ResponseSchema(name="creator", description="Who created the language")
]

# Create the parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get the format instructions
format_instructions = parser.get_format_instructions()

# Create prompt template with format instructions using partial variables
prompt = PromptTemplate(
    template="""Provide information about {language}.

{format_instructions}

Please provide your response:""",
    input_variables=["language"],
    partial_variables={"format_instructions": format_instructions}
)

# Create and run chain
chain = prompt | llm | parser

result = chain.invoke({"language": "Python"})

In [50]:
result

{'name': 'Python', 'year': '1991', 'creator': 'Guido van Rossum'}

## Runnable Sequence

In [16]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence,RunnableLambda

# Define the prompts
prompt1 = PromptTemplate(input_variables=["topic"], template="Explain {topic} in simple terms.")
prompt2 = PromptTemplate(input_variables=["text"], template="Summarize this {text} in three bullet points.")


chain = (
    prompt1 
    | llm 
    | RunnableLambda(lambda x: {
        "summary": (prompt2 | llm).invoke({"text": x.content}),
        "original": x.content
    })
)

result = chain.invoke({"topic": "quantum computing"})

# Output the result
print("Summary:", result["summary"])
print("Original Explanation:", result["original"])


Summary: content='* **Qubits and Superposition:** Unlike regular bits which are either 0 or 1, qubits can exist in a superposition, being both 0 and 1 simultaneously, like a spinning coin before it lands.\n* **Entanglement:** Multiple qubits can be linked through entanglement, meaning their states are correlated, even when separated.  Knowing the state of one entangled qubit reveals information about the others.\n* **Quantum Speedup:**  These properties allow quantum computers to explore many possibilities at once, making them potentially much faster than classical computers for specific complex tasks like drug discovery, materials science, financial modeling, and cryptography.\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-17c44ec3-5cb5-4e2c-be4c-0b246e3182aa-0' usage_metadata={'input_tokens': 361, 'output_tokens': 131, 'total_tokens': 492, 'input_token_details': {'cache_re

In [89]:
import json
from langchain_core.output_parsers import JsonOutputParser


explaination_prompt=PromptTemplate(input_variables=["topic"],template="""Provide a detailed explanation of the following topic: {topic}
    Your explanation should be comprehensive and clear.""")

key_points_prompt=PromptTemplate(input_variables=["explanation"],template="""Given the following explanation, extract 3-5 key points that capture the most important aspects:
    
    {explanation}
    
    Format your response as a JSON object with a single key 'key_points' containing an array of strings.""")


class KeyPointsOutputParser(JsonOutputParser):
    def parse(self,text):
        try:
            parsed=json.loads(text)
            return parsed["key_points"]
        except:
            return []



explaination_chain=explaination_prompt|llm
key_points_chain=key_points_prompt|llm|KeyPointsOutputParser()


explanation=explaination_chain.invoke({"topic":"GenAI"})

key_points=key_points_chain.invoke({"explanation":explanation.content})
        

In [96]:
valid=[x for x in key_points["key_points"]][:2]

In [97]:
valid

['GenAI creates new content like text, images, audio, and code by learning patterns from training data, unlike traditional AI that classifies or predicts.',
 'Key GenAI models include GANs (competing generator and discriminator), VAEs (latent space representation), Transformers (attention mechanism), and Diffusion Models (noise reversal).']

In [111]:
from langchain_core.runnables import Runnable

class SafeRun(Runnable):
    def __init__(self,func) -> None:
        self.func=func
    def invoke(self,input_data,config=None):
        try:
            return self.func(input_data)
        except Exception as e:
            print(f"Error in Runnable: {e}")
            return None
    
def numerical(input_data):
    if isinstance(input_data, dict):
        number = input_data.get("number")
        try:
            if int(number) > 10 or int(number) < 0:
                raise ValueError("Input must be between 0 and 10")
            return input_data  # Return original dict to maintain chain flow
        except ValueError as e:
            raise ValueError(f"Invalid input: {e}")
def numerical_2(message):
    try:
        if hasattr(message, 'content'):  # If it's an AIMessage
            content = message.content
        else:
            content = str(message)
            
        result = int(content)  # Try to convert to int
        if result > 0:  # Add any validation you need
            return str(result)  # Convert back to string for chain output
        else:
            raise ValueError("Output must be a positive integer")
    except ValueError as e:
        raise ValueError(f"Invalid output: {e}")


prompt=PromptTemplate(input_variables=["number"],template="""
                        Multiply the number {number} with any number of your choice and return the product alone
                        IMPORTANT:Just respond with the result do not add anything
                      """)
input_check=SafeRun(numerical)
output_check=SafeRun(numerical_2)

chain=input_check|prompt|llm|output_check

chain.invoke({"number":"10"})

Error in Runnable: Invalid input: Input must be between 0 and 10
Error in Runnable: Invalid output: invalid literal for int() with base 10: 'None\n'
